In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-05-03 19:59:08.382333: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 19:59:08.611745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with h5py.File('/eos/home-m/mmcohen/ntuples/AOD_EB_ntuples_04-04-2024.h5', 'r') as hf:
    HLT_jets = hf['HLT_jets'][:]
    L1_jFexSR_jets = hf['L1_jFexSR_jets'][:]
    L1_jFexLR_jets = hf['L1_jFexLR_jets'][:]
    HLT_electrons = hf['HLT_electrons'][:]
    LRT_electrons = hf['LRT_electrons'][:]
    L1_egammas = hf['L1_egammas'][:]
    HLT_muons = hf['HLT_muons'][:]
    LRT_muons = hf['LRT_muons'][:]
    L1_muons = hf['L1_muons'][:]
    L1_eFex_taus = hf['L1_eFex_taus'][:]
    L1_jFex_taus = hf['L1_jFex_taus'][:]
    HLT_photons = hf['HLT_photons'][:]
    HLT_MET = hf['HLT_MET'][:].reshape(-1, 1, 4)  # Broadcasting MET
    L1_MET = hf['L1_MET'][:].reshape(-1, 1, 4)
    #print(L1_MET.shape)
    #print(L1_MET[0,:,:])
    L1_MET_fixed=np.zeros((997315, 1, 3))
    for i,event in enumerate(L1_MET):
        L1_MET_fixed[i,0,0:2]=event[0,0:2]
        L1_MET_fixed[i,0,2]=event[0,3]
    #print(L1_MET_fixed.shape)
    #print(L1_MET_fixed[0,:,:])
    L1_MET=L1_MET_fixed
    pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
    pass_HLT_unprescaled = hf["pass_HLT_unprescaled"][:]
    EB_weights = hf["EB_weights"][:]


Topo_2A=[L1_muons,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_egammas]
Topo_2A_labels=['L1_muons','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_egammas']
Topo_2B=[L1_egammas,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_jFex_taus]
Topo_2B_labels=['L1_egammas','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_jFex_taus']
Topo_3A=[L1_egammas,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET]
Topo_3A_labels=['L1_egammas','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET']
#Topo_3B=[L1_muons,L1_egamma,L1_eFex_taus]


Topo_2A_concat=Topo_2A[0]
for i in Topo_2A[1:]:
    Topo_2A_concat=np.append(Topo_2A_concat,i,axis=1)
Topo_2A_flat=np.reshape(Topo_2A_concat,(-1,111))

Topo_2B_concat=Topo_2B[0]
for i in Topo_2B[1:]:
    Topo_2B_concat=np.append(Topo_2B_concat,i,axis=1)
Topo_2B_flat=np.reshape(Topo_2B_concat,(-1,75))

Topo_3A_concat=Topo_3A[0]
for i in Topo_3A[1:]:
    Topo_3A_concat=np.append(Topo_3A_concat,i,axis=1)
Topo_3A_flat=np.reshape(Topo_3A_concat,(-1,93))

In [3]:
Topo_2A_test=Topo_2A_flat[0:450000,:]
Topo_2B_test=Topo_2B_flat[0:450000,:]
Topo_3A_test=Topo_3A_flat[0:450000,:]

Topo_t_weights=EB_weights[0:450000]

In [4]:
 
nans=np.isnan(Topo_2A_test)
a=np.where(nans==True)[0]
for i in a:
    Topo_2A_test[i,92]=0


nans=np.isnan(Topo_2B_test)
b=np.where(nans==True)[0]
for i in b:
    Topo_2B_test[i,56]=0

nans=np.isnan(Topo_3A_test)
c=np.where(nans==True)[0]
for i in c:
    Topo_3A_test[i,92]=0

In [5]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var',kernel_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
 

In [6]:
#TOPO 2A Trainer

beta=0.85
T2A_enc=make_encoder(111,32,16,3)
T2A_dec=make_decoder(111,32,16,3)
T2A=VAE_Model(T2A_enc,T2A_dec)
T2A.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2A.fit(x=Topo_2A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_Version_1/',save_format='tf')

Epoch 1/100
4050/4050 [==============================] - 13s 3ms/step - loss: 251.3767 - reco_loss: 223.9196 - kl_loss: 13.6785 - val_loss: 1627.1548 - val_reco_loss: 78.8931 - val_kl_loss: 1548.2617 - lr: 0.0010
Epoch 2/100
4050/4050 [==============================] - 11s 3ms/step - loss: 85.1176 - reco_loss: 80.6436 - kl_loss: 4.0960 - val_loss: 1147.9344 - val_reco_loss: 73.3374 - val_kl_loss: 1074.5970 - lr: 0.0010
Epoch 3/100
4050/4050 [==============================] - 12s 3ms/step - loss: 78.2637 - reco_loss: 74.8025 - kl_loss: 3.3536 - val_loss: 1019.4985 - val_reco_loss: 73.8683 - val_kl_loss: 945.6302 - lr: 0.0010
Epoch 4/100
4050/4050 [==============================] - 11s 3ms/step - loss: 75.9147 - reco_loss: 72.8578 - kl_loss: 3.0010 - val_loss: 931.5119 - val_reco_loss: 72.9152 - val_kl_loss: 858.5967 - lr: 0.0010
Epoch 5/100
4050/4050 [==============================] - 11s 3ms/step - loss: 75.9415 - reco_loss: 73.0876 - kl_loss: 2.8352 - val_loss: 934.2949 - val_reco_los

In [7]:
#TOPO 2B Trainer

beta=0.85
T2B_enc=make_encoder(75,32,16,3)
T2B_dec=make_decoder(75,32,16,3)
T2B=VAE_Model(T2B_enc,T2B_dec)
T2B.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2B.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2B.fit(x=Topo_2B_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2B.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2B/Topo_2B_Version_1/',save_format='tf')

Epoch 1/100
4050/4050 [==============================] - 13s 3ms/step - loss: 414.1984 - reco_loss: 341.4351 - kl_loss: 30.9818 - val_loss: 2138.2222 - val_reco_loss: 107.7222 - val_kl_loss: 2030.5000 - lr: 0.0010
Epoch 2/100
4050/4050 [==============================] - 10s 2ms/step - loss: 109.3701 - reco_loss: 102.8733 - kl_loss: 5.8151 - val_loss: 1508.0205 - val_reco_loss: 101.6246 - val_kl_loss: 1406.3959 - lr: 0.0010
Epoch 3/100
4050/4050 [==============================] - 10s 2ms/step - loss: 100.2187 - reco_loss: 95.6897 - kl_loss: 4.3455 - val_loss: 1216.7061 - val_reco_loss: 103.3426 - val_kl_loss: 1113.3634 - lr: 0.0010
Epoch 4/100
4050/4050 [==============================] - 10s 3ms/step - loss: 100.6840 - reco_loss: 96.7336 - kl_loss: 3.9211 - val_loss: 1151.1348 - val_reco_loss: 96.3285 - val_kl_loss: 1054.8063 - lr: 0.0010
Epoch 5/100
4050/4050 [==============================] - 10s 2ms/step - loss: 99.6992 - reco_loss: 96.1128 - kl_loss: 3.4704 - val_loss: 1048.7162 - v

In [8]:
#TOPO 3A Trainer

beta=0.85
T3A_enc=make_encoder(93,32,16,3)
T3A_dec=make_decoder(93,32,16,3)
T3A=VAE_Model(T3A_enc,T3A_dec)
T3A.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T3A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T3A.fit(x=Topo_3A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T3A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_3A/Topo_3A_Version_1/',save_format='tf')

Epoch 1/100
4050/4050 [==============================] - 13s 3ms/step - loss: 436.6562 - reco_loss: 382.2867 - kl_loss: 22.7949 - val_loss: 1841.6810 - val_reco_loss: 92.3714 - val_kl_loss: 1749.3096 - lr: 0.0010
Epoch 2/100
4050/4050 [==============================] - 10s 3ms/step - loss: 102.8837 - reco_loss: 97.3162 - kl_loss: 5.2234 - val_loss: 1409.0919 - val_reco_loss: 88.7039 - val_kl_loss: 1320.3881 - lr: 0.0010
Epoch 3/100
4050/4050 [==============================] - 10s 2ms/step - loss: 93.1751 - reco_loss: 88.8617 - kl_loss: 4.1834 - val_loss: 1248.2759 - val_reco_loss: 87.4201 - val_kl_loss: 1160.8558 - lr: 0.0010
Epoch 4/100
4050/4050 [==============================] - 11s 3ms/step - loss: 88.1608 - reco_loss: 84.4596 - kl_loss: 3.6267 - val_loss: 1139.9036 - val_reco_loss: 87.6040 - val_kl_loss: 1052.2996 - lr: 0.0010
Epoch 5/100
4050/4050 [==============================] - 11s 3ms/step - loss: 87.9468 - reco_loss: 84.5207 - kl_loss: 3.3679 - val_loss: 1054.1940 - val_rec